In [76]:
import time
import logging
import requests
import pandas as pd

In [77]:
url = 'https://velox.horse/velox/api/horses/'
def get_url_data(api_url: str) -> pd.DataFrame:
    data = []
    url = api_url
    while url:
        logging.info(f'fetching url: {url}')
        resp = requests.get(url).json()
        results = resp['results']
        url = resp['next']
        for item in results:
            data.append(item)
        time.sleep(0.1)
    df = pd.DataFrame(data)
    return df

df = get_url_data(url)

In [78]:
# df[~df['date_last_start'].isnull()]
# df.iloc[6670].at['date_last_start'] == None
# df.to_csv('horses_duplicates.csv', index=False)
df[(df['date_last_start'].isnull()) & (df['measures'].map(len) == 0)]

,id,name,type,sex,date_of_birth,starts,date_last_start,country,tf_reg,race_rating,...,size1,class1,class2,class3,created_at,updated_at,update_days,start_days,days_old,measures
209,708,Stage Luck (USA),Flat,Female,2004-02-21,7.0,None,,4004998,106.00,...,CT,CC,GG,CC,2022-01-13T17:45:54.999961Z,2022-01-13T17:45:54.999979Z,22,0,6558,[]
210,709,Coco Belle (USA),Flat,Female,2004-02-26,26.0,None,,4010167,128.00,...,TT,AA,GG,AA,2022-01-13T17:45:55.003630Z,2022-01-13T17:45:55.003649Z,22,0,6553,[]
211,710,Cotton Blossom (USA),Flat,Female,2004-02-26,11.0,None,,4014427,116.00,...,CT,AC,GA,AC,2022-01-13T17:45:55.007037Z,2022-01-13T17:45:55.007055Z,22,0,6553,[]
212,711,Marietta (USA),Flat,Female,2004-03-30,10.0,None,,4013130,99.00,...,TT,CC,GG,AC,2022-01-13T17:45:55.010931Z,2022-01-13T17:45:55.010955Z,22,0,6520,[]
213,712,Pay The Man (USA),Flat,Female,2004-04-26,76.0,None,,4036749,104.00,...,TT,CC,GA,CC,2022-01-13T17:45:55.014382Z,2022-01-13T17:45:55.014400Z,22,0,6493,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8128,10557,Yellowstone Girl (USA),Flat,Female,2018-02-19,NaN,None,North America,,None,...,,,,,2022-02-01T20:12:47.576917Z,2022-02-02T12:34:17.851569Z,2,0,1446,[]
8129,10577,Zechariah (Ire),Flat,Male,2019-02-25,NaN,None,Europe,,None,...,,,,,2022-02-01T20:12:47.653262Z,2022-02-02T12:34:20.311758Z,2,0,1075,[]
8130,10586,Zisisit (IRE),Flat,Female,2019-04-04,NaN,None,Europe,,None,...,,,,,2022-02-01T20:12:47.686615Z,2022-02-02T12:34:21.416977Z,2,0,1037,[]
8131,10605,dk-test,Flat,Male,2022-02-01,0.0,None,None,None,0.00,...,None,None,None,None,2022-02-03T10:48:41.959239Z,2022-02-03T10:48:41.959267Z,1,0,3,[]


In [79]:
def remove_duplicates(dfx):
    total_len = len(dfx)
    if total_len > 1:
        dfr = dfx[(dfx['date_last_start'].isnull()) & (dfx['measures'].map(len) == 0)]
        if len(dfr) == total_len:
            # print('removing random ')
            ids = dfr['id'].to_list()
            ids_remove = ids[1:]
            # print(ids_remove)
            return ids_remove

        df_with_measures = dfx[dfx['measures'].map(len) > 0]
        df_no_measures = dfx[dfx['measures'].map(len) == 0]
        # print(f'with measure {len(df_with_measures)}, no measure {len(df_no_measures)}')
        if len(df_with_measures) == 1:
            # print('keeping one with measures')
            ids_remove = df_no_measures['id'].to_list()
            # print(ids_remove)
            return ids_remove

        df_with_start_date = dfx[~dfx['date_last_start'].isnull()]
        df_no_start_date = dfx[dfx['date_last_start'].isnull()]
        if len(df_with_start_date) == 1:
            ids_remove = df_no_start_date['id'].to_list()
            return ids_remove
        print('unknown', dfx)








dfg = df.groupby('name').apply(remove_duplicates)

# dfg[dfg['data']>3]

In [80]:
df_remove = dfg[~dfg.isnull()]
# df_remove.to_csv('to_remove.csv')
df_remove.to_json('to_remove1.json')